# Debugging using Large Language Models (LLMs) 1

In the code cell below, you will find a function that has its description and comments removed. The function is related to mobile robotics. It is your task to find out, what the function actually does in order to successfully call the function in your own program.

### Understanding the Function

First, open [ChatGPT](https://chat.openai.com/). You can initiate a conversation about this function by clicking the *New chat* button in the upper left corner and copying the function into the text input field. Additionally, ask the LLM what this Python function does.

*Hint: You can run notebook cells by clicking inside them and pressing the play button in the top bar. Alternatively, you can use the `Shift`+`Enter` shortcut to run the currently selected cell.*

In [2]:
import numpy as np

def forwardKinematics(vt, wt, lastPose, dt, dtype=np.float64, addNoise=True):
    if not isinstance(lastPose, np.ndarray): 
        lastPose = np.array(lastPose, dtype=dtype)
        assert lastPose.shape == (3,), "Received pose in wrong format! Pose needs to be provided in form [x,y,theta]!"
    if wt == 0: wt = np.finfo(dtype).tiny
    if addNoise == True:
        wt *= 1 + (np.random.random()-0.5)/20 # -> Error between +/- 2.5%
        vt *= 1 + (np.random.random()-0.5)/20 # -> Error between +/- 2.5%
        dt *= 1 + (np.random.random()-0.5)/20 # -> Error between +/- 2.5%
    vtwt = vt/wt
    _, _, theta = lastPose
    return lastPose + np.array([
        -vtwt*np.sin(theta) + vtwt*np.sin(theta + (wt*dt)),
        vtwt*np.cos(theta) - vtwt*np.cos(theta + (wt*dt)),
        wt*dt
    ], dtype=dtype)

##### Response:

<div class="llmresponse">
This Python function appears to implement the forward kinematics for a differential drive robot. The purpose of forward kinematics is to compute the robot's pose (position and orientation) in the world based on its velocity and angular velocity inputs.
</div>

# 

The LLM provides a description of what the function implements. You can keep the conversation up, by asking follow-up questions. For example, let's ask it, what the term *forward kinematics* means in context of mobile robots! You should get a response similar response as shown in the next cell.

##### Response:

<div class="llmresponse">
In the context of mobile robots, forward kinematics specifically refers to determining the position and orientation of the robot in its environment based on its motion parameters, typically linear and angular velocities. Mobile robots, such as differential drive robots or holonomic robots, use their wheels or other locomotion mechanisms to move around.
</div>

# 

Apart from asking about the a function's theoretical background, you can also ask the LLM questions about the code. For example, let's ask it what this function's return value looks like.

##### Response:

<div class="llmresponse">
The return value of the forwardKinematics function is a NumPy array representing the updated pose of the robot after the specified motion. The array has the same shape and data type as the input lastPose.

The pose array is calculated based on the differential drive kinematics equations, incorporating the provided linear velocity (vt), angular velocity (wt), the last known pose (lastPose), and the time step (dt). The resulting pose represents the new position and orientation of the robot after the simulated motion.

For example, if you call the function like this:


resulting_pose = forwardKinematics(1.0, 0.5, np.array([0.0, 0.0, 0.0]), 0.1)


The resulting_pose might be a NumPy array with values that reflect the updated position and orientation of the robot after moving forward with a linear velocity of 1.0, an angular velocity of 0.5, starting from an initial pose of [0.0, 0.0, 0.0], and a time step of 0.1. The specific values will depend on the input parameters and any random noise introduced if addNoise is set to True.
</div>

# 

While the answer tells us that the return value is a NumPy array, it does not tell us what shape this array will take. Therefore, we need to be more specific with our questions. When asked how long the returned array is, we finally get the response we were looking for:

##### Response:

<div class="llmresponse">
The returned array has the same length as the lastPose array, which is 3 in this case. The lastPose array represents the pose of the robot in the form [x, y, theta], where:

- x: The x-coordinate of the robot's position.
- y: The y-coordinate of the robot's position.
- theta: The orientation of the robot.
  
</div>

# 

Now that we somewhat understand the provided code's functionality, we can ask the LLM to add code comments in order to make the function easier to understand.

##### Prompt:

<div class="llmresponse">
Can you add code comments to the function to make it easier to understand?
</div>

##### Response:

<div class="llmresponse">
Certainly! Here's the original function with added comments to explain each section:
</div>

```python
def forwardKinematics(vt, wt, lastPose, dt, dtype=np.float64, addNoise=True):
    """
    Calculate forward kinematics for a differential drive robot.

    Parameters:
    - vt: Linear velocity of the robot.
    - wt: Angular velocity of the robot.
    - lastPose: Last known pose of the robot, [x, y, theta].
    - dt: Time step.
    - dtype: Data type for numerical precision (default is np.float64).
    - addNoise: Boolean flag indicating whether to add process noise (default is True).

    Returns:
    Updated pose of the robot after forward kinematics, [x, y, theta].
    """
    # Check input formatting
    if not isinstance(lastPose, np.ndarray): 
        lastPose = np.array(lastPose, dtype=dtype)
        assert lastPose.shape == (3,), "Received pose in wrong format! Pose needs to be provided in form [x, y, theta]!"

    # Set omega to smallest possible nonzero value in case it is zero to avoid division by zero
    if wt == 0:
        wt = np.finfo(dtype).tiny

    # Add optional process noise
    if addNoise:
        wt *= 1 + (np.random.random() - 0.5) / 20  # Error between +/- 2.5%
        vt *= 1 + (np.random.random() - 0.5) / 20  # Error between +/- 2.5%
        dt *= 1 + (np.random.random() - 0.5) / 20  # Error between +/- 2.5%

    # Calculate the ratio of linear velocity to angular velocity
    vtwt = vt / wt

    # Extract the theta value from the last pose
    _, _, theta = lastPose

    # Calculate the new pose based on differential drive kinematics equations
    updatedPose = lastPose + np.array([
        -vtwt * np.sin(theta) + vtwt * np.sin(theta + (wt * dt)),
        vtwt * np.cos(theta) - vtwt * np.cos(theta + (wt * dt)),
        wt * dt
    ], dtype=dtype)

    return updatedPose
```

<div class="llmresponse">
These comments should help clarify the purpose and functionality of each part of the function.
</div>

# 

### Examining the function's interface

Now we have a basic understanding of what the function actually does. The next step is to find out, how we successfully call the function. Directly asking the LLM, how to call the function gives us the following answer:

##### Response:

<div class="llmresponse">
The function forwardKinematics takes the following inputs:

1. vt: Linear velocity of the robot.
2. wt: Angular velocity of the robot.
3. lastPose: The last known pose of the robot, in the form of a NumPy array with shape (3,) representing [x, y, theta].
4. dt: Time step or duration of the motion.
5. dtype: Data type for NumPy arrays (default is np.float64).
6. addNoise: A boolean flag indicating whether to add random noise to the velocities and time step (default is True).

You would call the function like this:
</div>

```python
resulting_pose = forwardKinematics(vt_value, wt_value, last_pose_array, dt_value)
```
<div class="llmresponse">
Where vt_value, wt_value, last_pose_array, and dt_value are the actual values you want to use as input for the function.
</div>

# 

However, this does not tell us what values we can set as the function arguments. By asking how to format for example the *last_pose_array* argument, the LLM shows us, how to format the arguments:

##### Response:

<div class="llmresponse">

To initialize the last_pose_array, you need to provide the initial pose of the robot in the form of [x, y, theta]. You can create a NumPy array representing the initial pose like this:
</div>

```python
import numpy as np

initial_pose = [x_value, y_value, theta_value]
last_pose_array = np.array(initial_pose, dtype=np.float64)
```

<div class="llmresponse">
Replace x_value, y_value, and theta_value with the actual initial values for the robot's position and orientation. For example:
</div>

```python
x_value = 0.0
y_value = 0.0
theta_value = 0.0

last_pose_array = np.array([x_value, y_value, theta_value], dtype=np.float64)
```

<div class="llmresponse">
Now, you can use last_pose_array as an argument when calling the forwardKinematics function.
</div>

# 

### Exercise: Calling the function

Complete the next code cell by successfully calling the function *fowardKinematics* with arbitrary arguments and print out the return value. Let the LLM guide you in what values are provided as the function's arguments!

In [ ]:
import numpy as np

def forwardKinematics(vt, wt, lastPose, dt, dtype=np.float64, addNoise=True):
    if not isinstance(lastPose, np.ndarray): 
        lastPose = np.array(lastPose, dtype=dtype)
        assert lastPose.shape == (3,), "Received pose in wrong format! Pose needs to be provided in form [x,y,theta]!"
    if wt == 0: wt = np.finfo(dtype).tiny
    if addNoise == True:
        wt *= 1 + (np.random.random()-0.5)/20 # -> Error between +/- 2.5%
        vt *= 1 + (np.random.random()-0.5)/20 # -> Error between +/- 2.5%
        dt *= 1 + (np.random.random()-0.5)/20 # -> Error between +/- 2.5%
    vtwt = vt/wt
    _, _, theta = lastPose
    return lastPose + np.array([
        -vtwt*np.sin(theta) + vtwt*np.sin(theta + (wt*dt)),
        vtwt*np.cos(theta) - vtwt*np.cos(theta + (wt*dt)),
        wt*dt
    ], dtype=dtype)


#############################################
####    Your function call goes here:    ####



*Please ignore the next code cell. It is only used to style the LLM's responses throughout this notebook.*

In [2]:
%%html
<style>

div.llmresponse {
    width: 90%;
    padding-right: 28px;
    padding-left: 28px;
    color: gray;
    font-style: italic;
}
</style>